In [1]:
import os
import pathlib

import numpy as np
from torch_geometric.utils import from_networkx
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader

data = torch.load("datasets/data.pt")

In [35]:
class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes, num_neurons, network_size):
        super(GCN, self).__init__()

        self.conv_layers = torch.nn.ModuleList()
        self.conv_layers.append(GCNConv(num_features, num_neurons))
        for _ in range(network_size - 1):
            self.conv_layers.append(GCNConv(num_neurons, num_neurons))
        self.conv_layers.append(GCNConv(num_neurons, num_classes))

    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr

        for i, conv_layer in enumerate(self.conv_layers):
            if i != len(self.conv_layers) - 1:
                x = F.relu(conv_layer(x, edge_index, edge_weight))
            else:
                x = conv_layer(x, edge_index, edge_weight)

        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(data.num_node_features, int(data.y.max() + 2), 64, 3).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0134, weight_decay=0.000)

In [29]:
model.train()
for epoch in range(500):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    model.eval()
    pred = model(data).argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())

    print("Epoch: " + str(epoch))
    print(f'Loss: {loss:.4f}, Accuracy: {acc:.4f}')

Epoch: 0
Loss: 1.2673, Accuracy: 0.4324
Epoch: 1
Loss: 19.8146, Accuracy: 0.4324
Epoch: 2
Loss: 8.1701, Accuracy: 0.5676
Epoch: 3
Loss: 2.4441, Accuracy: 0.5676
Epoch: 4
Loss: 2.9676, Accuracy: 0.5676
Epoch: 5
Loss: 1.5286, Accuracy: 0.4815
Epoch: 6
Loss: 0.7180, Accuracy: 0.4324
Epoch: 7
Loss: 1.2495, Accuracy: 0.4324
Epoch: 8
Loss: 1.1380, Accuracy: 0.4247
Epoch: 9
Loss: 0.7770, Accuracy: 0.5676
Epoch: 10
Loss: 0.7077, Accuracy: 0.5676
Epoch: 11
Loss: 0.8139, Accuracy: 0.5676
Epoch: 12
Loss: 0.8389, Accuracy: 0.5676
Epoch: 13
Loss: 0.7919, Accuracy: 0.5676
Epoch: 14
Loss: 0.7275, Accuracy: 0.5676
Epoch: 15
Loss: 0.6882, Accuracy: 0.5679
Epoch: 16
Loss: 0.6846, Accuracy: 0.4401
Epoch: 17
Loss: 0.6995, Accuracy: 0.4294
Epoch: 18
Loss: 0.7055, Accuracy: 0.4305
Epoch: 19
Loss: 0.7035, Accuracy: 0.4545
Epoch: 20
Loss: 0.6979, Accuracy: 0.5279
Epoch: 21
Loss: 0.6910, Accuracy: 0.5712
Epoch: 22
Loss: 0.6840, Accuracy: 0.5676
Epoch: 23
Loss: 0.6796, Accuracy: 0.5676
Epoch: 24
Loss: 0.6862, A

In [30]:
torch.save(model, "models/detect_surface_feature_model.pt")

In [31]:
saved_model = torch.load("models/detect_surface_feature_model.pt")

saved_model = model.to(device)
saved_model.eval()

GCN(
  (conv_layers): ModuleList(
    (0): GCNConv(3, 64)
    (1-2): 2 x GCNConv(64, 64)
    (3): GCNConv(64, 3)
  )
)

In [43]:
new_data = torch.load("datasets/data_test.pt")
new_data = new_data.to(device)
pred = saved_model(new_data).argmax(dim=1)
print(pred)
print(new_data.y)

# predict correct with all nodes (not just test mask)
correct = (pred == new_data.y).sum()
acc = int(correct) / int(new_data.y.shape[0])

print(f'Accuracy: {acc:.4f}')

tensor([1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
        1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        1, 0, 0, 0], device='cuda:0')
tensor([0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0,
  